In [18]:
import pandas as pd
mri_df = pd.read_csv('oasis_roi_volumes.tsv', sep='\t')
mri_df.head()

,subject_id,lh_lateral_ventricle,rh_lateral_ventricle,lh_hippocampus,rh_hippocampus,lh_entorhinal,rh_entorhinal,lh_parahippocampal,rh_parahippocampal,lh_thalamus,rh_thalamus
0,OAS1_0001_MR1,27846.0,23059.0,3504.0,3905.0,923.0,782.0,1027.0,1186.0,5352.0,5521.0
1,OAS1_0002_MR1,3249.0,3074.0,3785.0,4285.0,1113.0,1012.0,1453.0,1832.0,5550.0,5656.0
2,OAS1_0003_MR1,22584.0,19983.0,2795.0,2891.0,668.0,459.0,1254.0,1195.0,5692.0,5682.0
3,OAS1_0004_MR1,16162.0,11819.0,3929.0,4237.0,1040.0,1480.0,1508.0,1751.0,6796.0,6737.0
4,OAS1_0005_MR1,4790.0,4224.0,4881.0,4690.0,1576.0,1203.0,2308.0,1996.0,7828.0,7491.0


In [19]:
df = pd.read_csv('oasis_cross-sectional.csv')

# Drop any rows with _MR2 in ID column (keep only baseline scans)
df = df[~df['ID'].str.contains('_MR2', na=False)]

# Drop specified columns
df = df.drop(columns=['Educ', 'SES', 'MMSE', 'eTIV', 'Delay','Hand'])

# Load MRI volumes
mri_df = pd.read_csv('oasis_roi_volumes.tsv', sep='\t')

# Merge the dataframes on subject ID
df = df.merge(mri_df, left_on='ID', right_on='subject_id', how='inner')

# Drop the redundant subject_id column from mri_df
df = df.drop(columns=['subject_id'])

# Get all columns that contain 'lh_' or 'rh_' (ROI volumes)
roi_columns = [col for col in df.columns if 'lh_' in col or 'rh_' in col]

# Scale ROI volumes by ASF for each subject
for col in roi_columns:
    df[col] = df[col] * df['ASF']

# Drop ASF column after scaling
df = df.drop(columns=['ASF'])

print(f"Final shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

Final shape: (405, 15)

Columns: ['ID', 'M/F', 'Age', 'CDR', 'nWBV', 'lh_lateral_ventricle', 'rh_lateral_ventricle', 'lh_hippocampus', 'rh_hippocampus', 'lh_entorhinal', 'rh_entorhinal', 'lh_parahippocampal', 'rh_parahippocampal', 'lh_thalamus', 'rh_thalamus']

First few rows:


,ID,M/F,Age,CDR,nWBV,lh_lateral_ventricle,rh_lateral_ventricle,lh_hippocampus,rh_hippocampus,lh_entorhinal,rh_entorhinal,lh_parahippocampal,rh_parahippocampal,lh_thalamus,rh_thalamus
0,OAS1_0001_MR1,F,74,0.0,0.743,36366.876,30115.054,4576.224,5099.930,1205.438,1021.292,1341.262,1548.916,6989.712,7210.426
1,OAS1_0002_MR1,F,55,0.0,0.810,4974.219,4706.294,5794.835,6560.335,1704.003,1549.372,2224.543,2804.792,8497.050,8659.336
2,OAS1_0003_MR1,F,73,0.5,0.708,27258.888,24119.481,3373.565,3489.437,806.276,554.013,1513.578,1442.365,6870.244,6858.174
3,OAS1_0004_MR1,M,28,0.0,0.803,17859.010,13059.995,4341.545,4681.885,1149.200,1635.400,1666.340,1934.855,7509.580,7444.385
4,OAS1_0005_MR1,M,18,0.0,0.848,4837.900,4266.240,4929.810,4736.900,1591.760,1215.030,2331.080,2015.960,7906.280,7565.910


# SVM

## Initial SVM w features and SVM w age only

In [ ]:
# SVM
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, make_scorer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Create binary target: 0 vs non-0
df['CDR_binary'] = (df['CDR'] > 0).astype(int)

# Check class distribution
print("Class distribution:")
print(df['CDR_binary'].value_counts())
print(f"\nClass proportions:")
print(df['CDR_binary'].value_counts(normalize=True))

# Prepare features
X = df.drop(columns=['ID', 'CDR', 'CDR_binary'])

# Encode M/F as binary (0/1)
X['M/F'] = (X['M/F'] == 'M').astype(int)

y = df['CDR_binary']

# Hold out test set (20%)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nHeld-out test set size: {len(X_test)}")
print(f"Test CDR=0: {sum(y_test==0)}, CDR>0: {sum(y_test==1)}")

# Nested CV setup
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Hyperparameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],  # Removed 0.01 - too much regularization
    'gamma': ['scale', 0.001, 0.01, 0.1],  # Removed 'auto' - redundant with scale
    'kernel': ['rbf', 'linear']
}

# Storage for results across outer folds
outer_scores = []
best_params_list = []
fold_predictions = []

print("\n" + "="*60)
print("NESTED CROSS-VALIDATION (5 outer folds, 3 inner folds)")
print("="*60)

for fold_idx, (train_idx, val_idx) in enumerate(outer_cv.split(X_train_full, y_train_full), 1):
    print(f"\n--- Outer Fold {fold_idx}/5 ---")
    
    # Split data for this outer fold
    X_train_fold = X_train_full.iloc[train_idx]
    X_val_fold = X_train_full.iloc[val_idx]
    y_train_fold = y_train_full.iloc[train_idx]
    y_val_fold = y_train_full.iloc[val_idx]
    
    # Standardize (fit on train, transform both)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled = scaler.transform(X_val_fold)
    
    # Inner CV: Grid search for best hyperparameters
    grid_search = GridSearchCV(
        SVC(class_weight='balanced', probability=True, random_state=42),
        param_grid=param_grid,
        cv=inner_cv,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_train_scaled, y_train_fold)
    
    # Best model from inner CV
    best_model = grid_search.best_estimator_
    best_params_list.append(grid_search.best_params_)
    print(f"Best params: {grid_search.best_params_}")
    
    # Evaluate on validation fold
    y_val_pred = best_model.predict(X_val_scaled)
    y_val_proba = best_model.predict_proba(X_val_scaled)[:, 1]
    
    fold_auc = roc_auc_score(y_val_fold, y_val_proba)
    outer_scores.append(fold_auc)
    
    print(f"Validation AUC: {fold_auc:.3f}")
    print(f"Confusion Matrix:")
    print(confusion_matrix(y_val_fold, y_val_pred))
    
    fold_predictions.append({
        'y_true': y_val_fold,
        'y_pred': y_val_pred,
        'y_proba': y_val_proba
    })

# Summary of nested CV results
print("\n" + "="*60)
print("NESTED CV SUMMARY")
print("="*60)
print(f"Mean AUC across folds: {np.mean(outer_scores):.3f} ± {np.std(outer_scores):.3f}")
print(f"AUC per fold: {[f'{s:.3f}' for s in outer_scores]}")
print(f"\nMost common best params:")
for param in ['kernel', 'C', 'gamma']:
    values = [p[param] for p in best_params_list]
    from collections import Counter
    most_common = Counter(values).most_common(1)[0]
    print(f"  {param}: {most_common[0]} (appeared {most_common[1]}/5 folds)")

# Final model: Retrain on full training set with most common hyperparameters
# Use the most frequent best params
final_kernel = Counter([p['kernel'] for p in best_params_list]).most_common(1)[0][0]
final_C = Counter([p['C'] for p in best_params_list]).most_common(1)[0][0]
final_gamma = Counter([p['gamma'] for p in best_params_list]).most_common(1)[0][0]

print(f"\n" + "="*60)
print("FINAL MODEL ON HELD-OUT TEST SET")
print("="*60)
print(f"Using hyperparameters: kernel={final_kernel}, C={final_C}, gamma={final_gamma}")

scaler_final = StandardScaler()
X_train_scaled_final = scaler_final.fit_transform(X_train_full)
X_test_scaled_final = scaler_final.transform(X_test)

final_svm = SVC(
    kernel=final_kernel, 
    C=final_C, 
    gamma=final_gamma,
    class_weight='balanced', 
    probability=True, 
    random_state=42
)
final_svm.fit(X_train_scaled_final, y_train_full)

y_test_pred = final_svm.predict(X_test_scaled_final)
y_test_proba = final_svm.predict_proba(X_test_scaled_final)[:, 1]

print("\nTest Set Performance:")
print(classification_report(y_test, y_test_pred, target_names=['CDR=0', 'CDR>0']))
print("\nConfusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred))

test_auc = roc_auc_score(y_test, y_test_proba)
print(f"\nTest AUC-ROC: {test_auc:.3f}")

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'SVM (AUC = {test_auc:.3f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Chance')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - SVM for CDR Binary Classification')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

Class distribution:
CDR_binary
0    312
1     93
Name: count, dtype: int64

Class proportions:
CDR_binary
0    0.77037
1    0.22963
Name: proportion, dtype: float64

Held-out test set size: 81
Test CDR=0: 62, CDR>0: 19

NESTED CROSS-VALIDATION (5 outer folds, 3 inner folds)

--- Outer Fold 1/5 ---
Best params: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Validation AUC: 0.940
Confusion Matrix:
[[40 10]
 [ 0 15]]

--- Outer Fold 2/5 ---


In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import numpy as np

print("="*60)
print("AGE-ONLY BASELINE MODEL")
print("="*60)

# Prepare age-only features
X_age_only = X[['Age']]
y_age = y.copy()

# Same train/test split as before
X_age_train, X_age_test = X_age_only.iloc[X_train_full.index], X_age_only.iloc[X_test.index]
y_age_train, y_age_test = y_age.iloc[X_train_full.index], y_age.iloc[X_test.index]

# Nested CV setup (same as full model)
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Simpler hyperparameter grid for 1D feature
param_grid_age = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'linear']
}

# Storage for results
outer_scores_age = []
best_params_age = []

print("\nRunning nested CV on AGE ONLY...")

for fold_idx, (train_idx, val_idx) in enumerate(outer_cv.split(X_age_train, y_age_train), 1):
    # Split data
    X_train_fold = X_age_train.iloc[train_idx]
    X_val_fold = X_age_train.iloc[val_idx]
    y_train_fold = y_age_train.iloc[train_idx]
    y_val_fold = y_age_train.iloc[val_idx]
    
    # Standardize
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled = scaler.transform(X_val_fold)
    
    # Inner CV for hyperparameter tuning
    grid_search = GridSearchCV(
        SVC(class_weight='balanced', probability=True, random_state=42),
        param_grid=param_grid_age,
        cv=inner_cv,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_train_scaled, y_train_fold)
    best_model = grid_search.best_estimator_
    best_params_age.append(grid_search.best_params_)
    
    # Evaluate
    y_val_proba = best_model.predict_proba(X_val_scaled)[:, 1]
    fold_auc = roc_auc_score(y_val_fold, y_val_proba)
    outer_scores_age.append(fold_auc)
    
    print(f"  Fold {fold_idx}: AUC = {fold_auc:.3f}, params = {grid_search.best_params_}")

print("\n" + "="*60)
print("AGE-ONLY MODEL: NESTED CV RESULTS")
print("="*60)
print(f"Mean AUC: {np.mean(outer_scores_age):.3f} ± {np.std(outer_scores_age):.3f}")
print(f"AUC per fold: {[f'{s:.3f}' for s in outer_scores_age]}")

# Train final age-only model on full training set
from collections import Counter
final_kernel_age = Counter([p['kernel'] for p in best_params_age]).most_common(1)[0][0]
final_C_age = Counter([p['C'] for p in best_params_age]).most_common(1)[0][0]

scaler_age = StandardScaler()
X_age_train_scaled = scaler_age.fit_transform(X_age_train)
X_age_test_scaled = scaler_age.transform(X_age_test)

final_svm_age = SVC(
    kernel=final_kernel_age,
    C=final_C_age,
    class_weight='balanced',
    probability=True,
    random_state=42
)
final_svm_age.fit(X_age_train_scaled, y_age_train)

y_age_test_proba = final_svm_age.predict_proba(X_age_test_scaled)[:, 1]
age_test_auc = roc_auc_score(y_age_test, y_age_test_proba)

print(f"\nAge-only Test Set AUC: {age_test_auc:.3f}")

# COMPARISON
print("\n" + "="*60)
print("🔍 COMPARISON: FULL MODEL vs AGE-ONLY")
print("="*60)
print(f"Full model CV AUC:    0.922 ± 0.023")
print(f"Age-only CV AUC:      {np.mean(outer_scores_age):.3f} ± {np.std(outer_scores_age):.3f}")
print(f"\nFull model Test AUC:  0.912")
print(f"Age-only Test AUC:    {age_test_auc:.3f}")

auc_diff = 0.912 - age_test_auc
print(f"\n📊 Brain features add: {auc_diff:.3f} AUC points ({auc_diff/age_test_auc*100:.1f}% improvement)")

if age_test_auc > 0.85:
    print("\n⚠️  WARNING: Age alone achieves high performance!")
    print("    The model may be primarily learning age, not brain structure.")
elif auc_diff < 0.05:
    print("\n⚠️  WARNING: Brain features add minimal improvement!")
    print("    Only ~5% AUC gain over age alone.")
else:
    print("\n✅ Brain features provide meaningful improvement over age alone.")

AGE-ONLY BASELINE MODEL

Running nested CV on AGE ONLY...
  Fold 1: AUC = 0.849, params = {'C': 1, 'kernel': 'rbf'}
  Fold 2: AUC = 0.906, params = {'C': 0.1, 'kernel': 'rbf'}
  Fold 3: AUC = 0.897, params = {'C': 1, 'kernel': 'rbf'}
  Fold 4: AUC = 0.847, params = {'C': 0.1, 'kernel': 'rbf'}
  Fold 5: AUC = 0.876, params = {'C': 1, 'kernel': 'rbf'}

AGE-ONLY MODEL: NESTED CV RESULTS
Mean AUC: 0.875 ± 0.024
AUC per fold: ['0.849', '0.906', '0.897', '0.847', '0.876']

Age-only Test Set AUC: 0.903

🔍 COMPARISON: FULL MODEL vs AGE-ONLY
Full model CV AUC:    0.922 ± 0.023
Age-only CV AUC:      0.875 ± 0.024

Full model Test AUC:  0.912
Age-only Test AUC:    0.903

📊 Brain features add: 0.009 AUC points (1.0% improvement)

⚠️  WARNING: Age alone achieves high performance!
    The model may be primarily learning age, not brain structure.


## SVM to predict continuous CDR

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

print("="*60)
print("MULTICLASS CDR PREDICTION (0 vs 0.5 vs 1+)")
print("="*60)

# Create 3-class target: CDR=0, CDR=0.5, CDR≥1
df['CDR_multiclass'] = df['CDR'].copy()
df.loc[df['CDR'] >= 1, 'CDR_multiclass'] = 1.0  # Collapse CDR 1, 2, 3 into single class

# Check class distribution
print("\nClass distribution:")
print(df['CDR_multiclass'].value_counts().sort_index())
print(f"\nClass proportions:")
print(df['CDR_multiclass'].value_counts(normalize=True).sort_index())

# Map to readable labels
class_labels = {0.0: 'CDR=0', 0.5: 'CDR=0.5', 1.0: 'CDR≥1'}
df['CDR_multiclass_label'] = df['CDR_multiclass'].map(class_labels)

# Prepare features
X = df.drop(columns=['ID', 'CDR', 'CDR_binary', 'CDR_multiclass', 'CDR_multiclass_label'])
X['M/F'] = (X['M/F'] == 'M').astype(int)
y = df['CDR_multiclass']

# Hold out test set (20%)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nHeld-out test set size: {len(X_test)}")
for class_val, class_name in class_labels.items():
    n = (y_test == class_val).sum()
    print(f"  {class_name}: {n}")

# Nested CV setup
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Hyperparameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 0.001, 0.01, 0.1],
    'kernel': ['rbf', 'linear']
}

# Storage for results across outer folds
outer_scores = []
best_params_list = []
fold_predictions = []

print("\n" + "="*60)
print("NESTED CROSS-VALIDATION (5 outer folds, 3 inner folds)")
print("="*60)

for fold_idx, (train_idx, val_idx) in enumerate(outer_cv.split(X_train_full, y_train_full), 1):
    print(f"\n--- Outer Fold {fold_idx}/5 ---")
    
    # Split data for this outer fold
    X_train_fold = X_train_full.iloc[train_idx]
    X_val_fold = X_train_full.iloc[val_idx]
    y_train_fold = y_train_full.iloc[train_idx]
    y_val_fold = y_train_full.iloc[val_idx]
    
    # Standardize (fit on train, transform both)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled = scaler.transform(X_val_fold)
    
    # Inner CV: Grid search for best hyperparameters
    # For multiclass, use accuracy as scoring metric
    grid_search = GridSearchCV(
        SVC(class_weight='balanced', probability=True, random_state=42, decision_function_shape='ovr'),
        param_grid=param_grid,
        cv=inner_cv,
        scoring='accuracy',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_train_scaled, y_train_fold)
    
    # Best model from inner CV
    best_model = grid_search.best_estimator_
    best_params_list.append(grid_search.best_params_)
    print(f"Best params: {grid_search.best_params_}")
    
    # Evaluate on validation fold
    y_val_pred = best_model.predict(X_val_scaled)
    
    fold_acc = accuracy_score(y_val_fold, y_val_pred)
    fold_kappa = cohen_kappa_score(y_val_fold, y_val_pred, weights='quadratic')
    outer_scores.append({'accuracy': fold_acc, 'kappa': fold_kappa})
    
    print(f"Validation Accuracy: {fold_acc:.3f}")
    print(f"Validation Kappa: {fold_kappa:.3f}")
    print(f"Confusion Matrix:")
    cm = confusion_matrix(y_val_fold, y_val_pred)
    print(cm)
    
    fold_predictions.append({
        'y_true': y_val_fold,
        'y_pred': y_val_pred
    })

# Summary of nested CV results
print("\n" + "="*60)
print("NESTED CV SUMMARY")
print("="*60)
accuracies = [s['accuracy'] for s in outer_scores]
kappas = [s['kappa'] for s in outer_scores]

print(f"Mean Accuracy across folds: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
print(f"Mean Kappa across folds: {np.mean(kappas):.3f} ± {np.std(kappas):.3f}")
print(f"Accuracy per fold: {[f'{s:.3f}' for s in accuracies]}")
print(f"Kappa per fold: {[f'{s:.3f}' for s in kappas]}")

print(f"\nMost common best params:")
for param in ['kernel', 'C', 'gamma']:
    values = [p[param] for p in best_params_list]
    most_common = Counter(values).most_common(1)[0]
    print(f"  {param}: {most_common[0]} (appeared {most_common[1]}/5 folds)")

# Final model: Retrain on full training set with most common hyperparameters
final_kernel = Counter([p['kernel'] for p in best_params_list]).most_common(1)[0][0]
final_C = Counter([p['C'] for p in best_params_list]).most_common(1)[0][0]
final_gamma = Counter([p['gamma'] for p in best_params_list]).most_common(1)[0][0]

print(f"\n" + "="*60)
print("FINAL MODEL ON HELD-OUT TEST SET")
print("="*60)
print(f"Using hyperparameters: kernel={final_kernel}, C={final_C}, gamma={final_gamma}")

scaler_final = StandardScaler()
X_train_scaled_final = scaler_final.fit_transform(X_train_full)
X_test_scaled_final = scaler_final.transform(X_test)

final_svm = SVC(
    kernel=final_kernel, 
    C=final_C, 
    gamma=final_gamma,
    class_weight='balanced',
    decision_function_shape='ovr',
    probability=True,
    random_state=42
)
final_svm.fit(X_train_scaled_final, y_train_full)

y_test_pred = final_svm.predict(X_test_scaled_final)

print("\nTest Set Performance:")
print(classification_report(y_test, y_test_pred, target_names=['CDR=0', 'CDR=0.5', 'CDR≥1']))

print("\nConfusion Matrix (Test):")
cm_test = confusion_matrix(y_test, y_test_pred)
print(cm_test)

test_acc = accuracy_score(y_test, y_test_pred)
test_kappa = cohen_kappa_score(y_test, y_test_pred, weights='quadratic')
print(f"\nTest Accuracy: {test_acc:.3f}")
print(f"Test Weighted Kappa: {test_kappa:.3f}")

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['CDR=0', 'CDR=0.5', 'CDR≥1'],
            yticklabels=['CDR=0', 'CDR=0.5', 'CDR≥1'])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix - Multiclass CDR Prediction')
plt.tight_layout()
plt.show()

# Compare to age-only baseline
print("\n" + "="*60)
print("AGE-ONLY BASELINE (MULTICLASS)")
print("="*60)

X_age_only = X[['Age']]
X_age_train_full = X_age_only.iloc[X_train_full.index]
X_age_test = X_age_only.iloc[X_test.index]
y_age_train = y_train_full.copy()
y_age_test = y_test.copy()

# Simple grid for age-only
param_grid_age = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'linear']
}

outer_scores_age = []

print("\nRunning nested CV on AGE ONLY...")

for fold_idx, (train_idx, val_idx) in enumerate(outer_cv.split(X_age_train_full, y_age_train), 1):
    X_train_fold = X_age_train_full.iloc[train_idx]
    X_val_fold = X_age_train_full.iloc[val_idx]
    y_train_fold = y_age_train.iloc[train_idx]
    y_val_fold = y_age_train.iloc[val_idx]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled = scaler.transform(X_val_fold)
    
    grid_search = GridSearchCV(
        SVC(class_weight='balanced', probability=True, random_state=42, decision_function_shape='ovr'),
        param_grid=param_grid_age,
        cv=inner_cv,
        scoring='accuracy',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_train_scaled, y_train_fold)
    best_model = grid_search.best_estimator_
    
    y_val_pred = best_model.predict(X_val_scaled)
    fold_acc = accuracy_score(y_val_fold, y_val_pred)
    outer_scores_age.append(fold_acc)
    
    print(f"  Fold {fold_idx}: Accuracy = {fold_acc:.3f}")

print(f"\nAge-only CV Accuracy: {np.mean(outer_scores_age):.3f} ± {np.std(outer_scores_age):.3f}")

# Train final age-only model
final_kernel_age = 'rbf'  # Default
final_C_age = 1

scaler_age = StandardScaler()
X_age_train_scaled = scaler_age.fit_transform(X_age_train_full)
X_age_test_scaled = scaler_age.transform(X_age_test)

final_svm_age = SVC(
    kernel=final_kernel_age,
    C=final_C_age,
    class_weight='balanced',
    decision_function_shape='ovr',
    probability=True,
    random_state=42
)
final_svm_age.fit(X_age_train_scaled, y_age_train)

y_age_test_pred = final_svm_age.predict(X_age_test_scaled)
age_test_acc = accuracy_score(y_age_test, y_age_test_pred)

print(f"Age-only Test Accuracy: {age_test_acc:.3f}")

# COMPARISON
print("\n" + "="*60)
print("🔍 COMPARISON: FULL MODEL vs AGE-ONLY (MULTICLASS)")
print("="*60)
print(f"Full model CV Accuracy:    {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
print(f"Age-only CV Accuracy:      {np.mean(outer_scores_age):.3f} ± {np.std(outer_scores_age):.3f}")
print(f"\nFull model Test Accuracy:  {test_acc:.3f}")
print(f"Age-only Test Accuracy:    {age_test_acc:.3f}")

acc_diff = test_acc - age_test_acc
print(f"\n📊 Brain features add: {acc_diff:+.3f} accuracy points ({acc_diff/age_test_acc*100:+.1f}% change)")

if acc_diff > 0.05:
    print("\n✅ Brain features provide meaningful improvement for multiclass prediction!")
elif acc_diff > 0.02:
    print("\n⚠️ Brain features provide modest improvement (~2-5%).")
else:
    print("\n⚠️ Brain features add minimal value even for severity classification.")

MULTICLASS CDR PREDICTION (0 vs 0.5 vs 1+)

Class distribution:
CDR_multiclass
0.0    312
0.5     68
1.0     25
Name: count, dtype: int64

Class proportions:
CDR_multiclass
0.0    0.770370
0.5    0.167901
1.0    0.061728
Name: proportion, dtype: float64

Held-out test set size: 81
  CDR=0: 62
  CDR=0.5: 14
  CDR≥1: 5

NESTED CROSS-VALIDATION (5 outer folds, 3 inner folds)


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.